In [1]:
import json
import cv2
import numpy as np

In [23]:
with open('.\\sample3\\1440.json') as json_file:
    json_f = json.load(json_file)
img = cv2.imread('.\\sample3\\frame1440.png',0)

In [24]:
arr = np.zeros((img.shape[0],img.shape[1],2))

In [25]:
arr.shape

(720, 1280, 2)

In [26]:
for dic in json_f:
    width = int(dic["width"] / 2)
    height = int(dic["height"] / 2)
    x = dic['src_x']
    y = dic['src_y']
    arr[y-height:y+height,x-width:x+width,0] = dic["dx"]
    arr[y-height:y+height,x-width:x+width,1] = dic["dy"]

In [49]:
# Visualize optical flow
# =======================
# Code modified from:
# https://github.com/tomrunia/OpticalFlow_Visualization/blob/master/flow_vis.py

import numpy as np


def make_colorwheel():
    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR
    colorwheel = np.zeros((ncols, 3))
    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.floor(255 * np.arange(0, RY) / RY)
    col = col + RY
    # YG
    colorwheel[col:col + YG, 0] = 255 - np.floor(255 * np.arange(0, YG) / YG)
    colorwheel[col:col + YG, 1] = 255
    col = col + YG
    # GC
    colorwheel[col:col + GC, 1] = 255
    colorwheel[col:col + GC, 2] = np.floor(255 * np.arange(0, GC) / GC)
    col = col + GC
    # CB
    colorwheel[col:col + CB, 1] = 255 - np.floor(255 * np.arange(CB) / CB)
    colorwheel[col:col + CB, 2] = 255
    col = col + CB
    # BM
    colorwheel[col:col + BM, 2] = 255
    colorwheel[col:col + BM, 0] = np.floor(255 * np.arange(0, BM) / BM)
    col = col + BM
    # MR
    colorwheel[col:col + MR, 2] = 255 - np.floor(255 * np.arange(MR) / MR)
    colorwheel[col:col + MR, 0] = 255
    return colorwheel
    #return np.flip(colorwheel,1)

def flow_compute_color(flow_u, flow_v):
    H, W = flow_u.shape[:2]
    flow_image = np.zeros((H, W, 3), np.uint8)

    colorwheel = make_colorwheel()  # shape [55x3]
    ncols = colorwheel.shape[0]

    rad = np.sqrt(np.square(flow_u) + np.square(flow_v))
    a = np.arctan2(- flow_v, - flow_u) / np.pi

    fk = (a + 1) / 2 * (ncols - 1) + 1
    k0 = np.floor(fk).astype(np.int32)
    k1 = k0 + 1
    k1[k1 == ncols] = 1
    f = fk - k0

    for i in range(colorwheel.shape[1]):

        tmp = colorwheel[:, i]
        col0 = tmp[k0] / 255.0
        col1 = tmp[k1] / 255.0
        col = (1 - f) * col0 + f * col1

        idx = (rad <= 1)
        col[idx] = 1 - rad[idx] * (1 - col[idx])
        col[~idx] = col[~idx] * 0.75   # out of range?

        ch_idx = i
        flow_image[:, :, ch_idx] = np.floor(255 * col)

    return flow_image


def flow2rgb(flow_uv):
    """Visualize optical flow.
    Parameters
    ----------
    flow_uv: numpy.ndarray, (H, W, 2), float
        Optical flow.
    Returns
    -------
    dst: numpy.ndarray
        RGB image.
    """
    assert flow_uv.ndim == 3, 'flow must be 3 dimensional'
    assert flow_uv.shape[2] == 2, 'flow must have shape (H, W, 2)'
    assert np.issubdtype(flow_uv.dtype, np.floating), \
        'float must be float type'

    flow_u = flow_uv[:, :, 0]
    flow_v = flow_uv[:, :, 1]

    rad = np.sqrt(np.square(flow_u) + np.square(flow_v))
    rad_max = np.max(rad)

    epsilon = 1e-5
    flow_u = flow_u / (rad_max + epsilon)
    flow_v = flow_v / (rad_max + epsilon)

    return flow_compute_color(flow_u, flow_v)

In [28]:
output = flow2rgb(arr)

In [29]:
cv2.imwrite('.\\sample3\\MV_v2.png',output) 

True

In [30]:
x = make_colorwheel().reshape((1,55,3))

In [31]:
cv2.imwrite('colorwheel.png',x) 

True

mmcv colorwheel

In [45]:
def make_color_wheel(bins=None):
    """Build a color wheel.
    Args:
        bins(list or tuple, optional): Specify the number of bins for each
            color range, corresponding to six ranges: red -> yellow,
            yellow -> green, green -> cyan, cyan -> blue, blue -> magenta,
            magenta -> red. [15, 6, 4, 11, 13, 6] is used for default
            (see Middlebury).
    Returns:
        ndarray: Color wheel of shape (total_bins, 3).
    """
    if bins is None:
        bins = [15, 6, 4, 11, 13, 6]
    assert len(bins) == 6

    RY, YG, GC, CB, BM, MR = tuple(bins)

    ry = [1, np.arange(RY) / RY, 0]
    yg = [1 - np.arange(YG) / YG, 1, 0]
    gc = [0, 1, np.arange(GC) / GC]
    cb = [0, 1 - np.arange(CB) / CB, 1]
    bm = [np.arange(BM) / BM, 0, 1]
    mr = [1, 0, 1 - np.arange(MR) / MR]

    num_bins = RY + YG + GC + CB + BM + MR

    color_wheel = np.zeros((3, num_bins), dtype=np.float32)

    col = 0
    for i, color in enumerate([ry, yg, gc, cb, bm, mr]):
        for j in range(3):
            color_wheel[j, col:col + bins[i]] = color[j]
        col += bins[i]

    return color_wheel.T

In [51]:
np.flip(make_colorwheel(),1)

array([[  0.,   0., 255.],
       [  0.,  17., 255.],
       [  0.,  34., 255.],
       [  0.,  51., 255.],
       [  0.,  68., 255.],
       [  0.,  85., 255.],
       [  0., 102., 255.],
       [  0., 119., 255.],
       [  0., 136., 255.],
       [  0., 153., 255.],
       [  0., 170., 255.],
       [  0., 187., 255.],
       [  0., 204., 255.],
       [  0., 221., 255.],
       [  0., 238., 255.],
       [  0., 255., 255.],
       [  0., 255., 213.],
       [  0., 255., 170.],
       [  0., 255., 128.],
       [  0., 255.,  85.],
       [  0., 255.,  43.],
       [  0., 255.,   0.],
       [ 63., 255.,   0.],
       [127., 255.,   0.],
       [191., 255.,   0.],
       [255., 255.,   0.],
       [255., 232.,   0.],
       [255., 209.,   0.],
       [255., 186.,   0.],
       [255., 163.,   0.],
       [255., 140.,   0.],
       [255., 116.,   0.],
       [255.,  93.,   0.],
       [255.,  70.,   0.],
       [255.,  47.,   0.],
       [255.,  24.,   0.],
       [255.,   0.,   0.],
 

In [50]:
make_colorwheel()

array([[255.,   0.,   0.],
       [255.,  17.,   0.],
       [255.,  34.,   0.],
       [255.,  51.,   0.],
       [255.,  68.,   0.],
       [255.,  85.,   0.],
       [255., 102.,   0.],
       [255., 119.,   0.],
       [255., 136.,   0.],
       [255., 153.,   0.],
       [255., 170.,   0.],
       [255., 187.,   0.],
       [255., 204.,   0.],
       [255., 221.,   0.],
       [255., 238.,   0.],
       [255., 255.,   0.],
       [213., 255.,   0.],
       [170., 255.,   0.],
       [128., 255.,   0.],
       [ 85., 255.,   0.],
       [ 43., 255.,   0.],
       [  0., 255.,   0.],
       [  0., 255.,  63.],
       [  0., 255., 127.],
       [  0., 255., 191.],
       [  0., 255., 255.],
       [  0., 232., 255.],
       [  0., 209., 255.],
       [  0., 186., 255.],
       [  0., 163., 255.],
       [  0., 140., 255.],
       [  0., 116., 255.],
       [  0.,  93., 255.],
       [  0.,  70., 255.],
       [  0.,  47., 255.],
       [  0.,  24., 255.],
       [  0.,   0., 255.],
 

In [47]:
make_color_wheel()

array([[1.        , 0.        , 0.        ],
       [1.        , 0.06666667, 0.        ],
       [1.        , 0.13333334, 0.        ],
       [1.        , 0.2       , 0.        ],
       [1.        , 0.26666668, 0.        ],
       [1.        , 0.33333334, 0.        ],
       [1.        , 0.4       , 0.        ],
       [1.        , 0.46666667, 0.        ],
       [1.        , 0.53333336, 0.        ],
       [1.        , 0.6       , 0.        ],
       [1.        , 0.6666667 , 0.        ],
       [1.        , 0.73333335, 0.        ],
       [1.        , 0.8       , 0.        ],
       [1.        , 0.8666667 , 0.        ],
       [1.        , 0.93333334, 0.        ],
       [1.        , 1.        , 0.        ],
       [0.8333333 , 1.        , 0.        ],
       [0.6666667 , 1.        , 0.        ],
       [0.5       , 1.        , 0.        ],
       [0.33333334, 1.        , 0.        ],
       [0.16666667, 1.        , 0.        ],
       [0.        , 1.        , 0.        ],
       [0.

In [38]:
cv2.imwrite('colorwheel_mmcv.jpg',x) 

True